<a href="https://colab.research.google.com/github/duyvm/funny_stuff_with_llm/blob/main/learning-rag%5CLangchain_Rag_Guide_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

- Install required packages
- Set environment variables
- Load model and vector db

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain[openai] langchain-core

In [ ]:
from google.colab import userdata
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = f"langchain-learning-rag"
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

llm = init_chat_model("gpt-4o-mini", model_provider="openai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# Preview

- Guide: [Build a Retrieval Augmented Generation (RAG) App: Part 1](https://python.langchain.com/docs/tutorials/rag/)

- Build an app that answers questions about the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng
. Allows us to ask questions about the contents of the post.

- Create a simple indexing pipeline and RAG chain

- Test around with another site, variants of question, multi languages

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# load and chunk the content of the blog
blog_url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

loader = WebBaseLoader(
    web_paths=(blog_url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

doc = loader.load()

In [ ]:
# split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(doc)

In [ ]:
# testing some facts
fabricated_fact = "Vo Minh Duy is a genius Vietnamese. He won 3 noble prizes in a row in Computer Science major. He graduated in Havard and now working in Marvel alliance. He is Iron Man's bff"
fabricated_doc = Document(page_content=fabricated_fact)
all_splits.append(fabricated_doc)

In [ ]:
# index chunk and store in db
_ = vector_store.add_documents(all_splits)

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# define state of langgraph
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
# define retrieva; step in graph
# the actual function that will call to retrieve documents in vector db
def retrieve(state: State) -> dict:
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

In [ ]:
# define generation step
def generate(state: State) -> dict:
    doc_content = "\n\n".join([doc.page_content for doc in state["context"]])
    prompt_with_context = prompt.invoke(input={"context": doc_content, "question": state["question"]})
    answer = llm.invoke(prompt_with_context)
    return {"answer": answer.content}

In [ ]:
# compile the graph and test
graph_builder = StateGraph(State).add_sequence([("1st", retrieve), ("2nd", generate)])
graph_builder.add_edge(START, "1st")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complicated task into smaller, manageable steps or subgoals. This can be achieved through various methods, including prompting language models, using task-specific instructions, or incorporating human inputs. Advanced techniques like Chain of Thought (CoT) and Tree of Thoughts further enhance this process by allowing iterative reasoning and exploration of multiple approaches.


In [ ]:
response = graph.invoke({"question": "Who is Vo Minh Duy?"})
print(response["answer"])

Vo Minh Duy is a Vietnamese genius who has won three Nobel Prizes in Computer Science. He graduated from Harvard and currently works with Marvel Alliance, where he is a close friend of Iron Man.


In [ ]:
response = graph.invoke({"question": "Any Vietnamese in the document?"})
print(response["answer"])

Yes, Vo Minh Duy is mentioned as a Vietnamese individual. He is described as a genius who won three Nobel Prizes in Computer Science.


In [ ]:
# some information not in our data store
response = graph.invoke({"question": "Who is Madam Luck?"})
print(response["answer"])

I don't know.


In [ ]:
response = graph.invoke({"question": "Tell me the greatness of Vo Minh Duy?"})
print(response["answer"])

Vo Minh Duy is recognized as a genius Vietnamese who achieved the remarkable feat of winning three Nobel Prizes in Computer Science. He graduated from Harvard University and currently works for Marvel Alliance, where he is noted as Iron Man's best friend. His achievements highlight his exceptional intellect and contributions to the field.


In [ ]:
response = graph.invoke({"question": "Cho tôi thông tin về ông Võ"})
print(response["answer"])

Ông Võ là Võ Minh Duy, một thiên tài người Việt Nam đã giành được 3 giải Nobel liên tiếp trong lĩnh vực Khoa học Máy tính. Ông tốt nghiệp từ Harvard và hiện đang làm việc tại Marvel Alliance, là bạn thân của Iron Man.


In [ ]:
response = graph.invoke({"question": "ボ・ミン・ズイさんについて教えてください"})
print(response["answer"])

ボ・ミン・ズイはベトナムの天才で、コンピュータサイエンスの分野で3回連続してノーベル賞を受賞しました。ハーバード大学を卒業後、現在はマーベル・アライアンスで働いており、アイアンマンの親友でもあります。


In [ ]:
response = graph.invoke({"question": "ボについて教えてください"})
print(response["answer"])

「ボ」についての情報は提供されていません。具体的な内容を知りたい場合は、もう少し詳しい質問をしてください。


Wrong answer. Provided information in not match. Need more information

In [ ]:
response = graph.invoke({"question": "ボさんについて教えてください"})
print(response["answer"])

ボは「魔王軍の（なんちゃって）幹部」であり、アンデッドの王・リッチーとして知られています。他にも多くの悪魔や幹部が登場しますが、ボ自身についての具体的な情報は記載されていません。ボに関する詳細は「ウィズ」を参照してください。


In [ ]:
response = graph.invoke({"question": "Vo Minh Duyさんについて教えてください"})
print(response["answer"])

Vo Minh Duyさんは、3つのノーベル賞を受賞したベトナム出身の天才で、コンピュータサイエンスを専攻しています。ハーバード大学を卒業し、現在はマーベルアライアンスで働いており、アイアンマンの親友として知られています。


In [ ]:
response = graph.invoke({"question": "Duy là ai thế?"})
print(response["answer"])

Duy is Vo Minh Duy, a genius Vietnamese who has won three Nobel Prizes in Computer Science. He graduated from Harvard and currently works at Marvel Alliance as Iron Man's best friend.


In [ ]:
response = graph.invoke({"question": "Duy là ai thế? Trả lời tôi bằng tiếng Việt"})
print(response["answer"])

Duy là Vo Minh Duy, một thiên tài người Việt Nam. Anh đã giành được 3 giải Nobel liên tiếp trong lĩnh vực Khoa học máy tính và hiện đang làm việc tại Marvel Alliance, là bạn thân của Iron Man.


In [ ]:
# testing not English document
# https://ja.wikipedia.org/wiki/%E3%81%93%E3%81%AE%E7%B4%A0%E6%99%B4%E3%82%89%E3%81%97%E3%81%84%E4%B8%96%E7%95%8C%E3%81%AB%E7%A5%9D%E7%A6%8F%E3%82%92!

# ask first
response = graph.invoke({"question": "Tell me about 'この素晴らしい世界に祝福を!'"})
print(response["answer"])

I don't know.


In [ ]:
# load new site, splint and index it
new_loader = WebBaseLoader(
    web_paths=("https://ja.wikipedia.org/wiki/%E3%81%93%E3%81%AE%E7%B4%A0%E6%99%B4%E3%82%89%E3%81%97%E3%81%84%E4%B8%96%E7%95%8C%E3%81%AB%E7%A5%9D%E7%A6%8F%E3%82%92!",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-content-ltr mw-parser-output", "mw-page-title-main")
        )
    )
)

new_doc = new_loader.load()

In [ ]:
new_splits = text_splitter.split_documents(new_doc)

_ = vector_store.add_documents(new_splits)

In [ ]:
# ask again
response = graph.invoke({"question": "Tell me about 'この素晴らしい世界に祝福を!'"})
print(response["answer"])

『この素晴らしい世界に祝福を！』は、暁なつめによる日本のライトノベルで、2013年から2023年にかけて刊行されました。元々は小説投稿サイト「小説家になろう」で連載されており、異世界ファンタジーとコメディの要素を含んでいます。また、アニメ化や劇場アニメの公開もされており、シリーズは累計1000万部を超える人気を誇っています。


In [ ]:
# ask again
response = graph.invoke({"question": "Tell me about 'この素晴らしい世界に祝福を!'. Answer in English"})
print(response["answer"])

"KonoSuba: God's Blessing on This Wonderful World!" is a Japanese light novel series written by Natsume Akatsuki, which started serialization in December 2012. It has been adapted into a manga and an anime series, with the first anime airing from January to March 2016 and a third season scheduled for 2024. The series is known for its adventure, fantasy, and comedy elements, and has gained significant popularity, selling over 10 million copies by November 2021.


In [ ]:
# ask again
response = graph.invoke({"question": "Tell me about 'Konosuba: God's Blessing on This Wonderful world'"})
print(response["answer"])

『この素晴らしい世界に祝福を！』（略称：このすば）は、暁なつめのライトノベルで、異世界に転生した高校生カズマと女神アクアを中心にしたコメディ冒険物語です。2016年からテレビアニメ化され、シリーズ累計発行部数は1000万部を超えています。物語は、カズマがアクアを連れて異世界で仲間と共に様々なトラブルに巻き込まれていく様子を描いています。


In [ ]:
# ask again with lot of un-use information
response = graph.invoke({"question": """
    Tell me about 'この素晴らしい世界に祝福を!'. Answer in English.
    By 2038 it will enter an aging-population period.
    Aging is one of the major challenges in population management, impacting economic growth, social welfare, labor, infrastructure design, and, especially, healthcare.
    The average life expectancy in Vietnam is high (74.7 years), but health is poor, with people spending 14 years suffering from illnesses, according to the ministry.
    Elderly people often suffer from many non-communicable diseases that require lifelong treatment, such as hypertension, cardiovascular issues, diabetes, and dementia.
    Their healthcare costs also rise, creating financial pressure on the health insurance system and government finances.
    Vietnam has over 1,300 public hospitals, more than 100 of which are central or provincial hospitals with geriatric departments, but there are fewer than 1,800 healthcare workers trained in geriatrics.
    The shortage of geriatricians and long-term care services is seen as a major challenge, Le Thanh Dung, director of the ministry's population department, said.
    In the event, the bill proposes a number of policies to develop human resources for senior healthcare, including offering scholarships and tuition support for people studying geriatrics, creating training programs for elderly care, funding community-based training for elderly healthcare workers, and encouraging organizations and individuals to provide scholarships and grants.
    The ministry also plans to fully subsidize health insurance for elderly people who lack cover, estimated at 5%.
    Many other countries are also addressing the aging issue.
    """})
print(response["answer"])

'この素晴らしい世界に祝福を!' (Kono Subarashii Sekai ni Shukufuku wo!) is a Japanese light novel series written by Natsume Akatsuki, which began serialization in 2013. The story follows a young man who is reincarnated in a fantasy world and is accompanied by various quirky characters, including a useless goddess. The series blends fantasy tropes with humor, featuring unique elements like vegetables escaping and absurd situations.


In [ ]:
# ask again
response = graph.invoke({"question": "English please"})
print(response["answer"])

I don't know.


# 🧠 Conclude

👎 In this example, we consider user's question is the whole information for searching in vector db. It can lead to retrieving incorrect informations if the actual information that we want is too small consider to the length of question.

  ↪ For example:

     ❓ quesion: `Tell me about 'この素晴らしい世界に祝福を!'. Answer in English. xxx`

     ❗ actual wanted information: `この素晴らしい世界に祝福を!`

     🤔 The problem is how to seperate the actual relevant information piece for searching with the rest in the question


👍 The app can understand multi-languages

👍 The app can pick the right piece of information from retrieved informations, or perform other task like summarize.

👍 The app can understand the variants of the question

🤔 The accuracy of answer depends a lot on retrieved informations. It can be tricky when stored information and provided information (in question) is not in the same language